# ◆ 통계적 검정 방법(t검정) 실습

## 패키지 Import

In [1]:
# 패키지 임포트
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

import pandas as pd # pandas package
import numpy as np # numpy package
import math
from scipy import stats # Scipy Package

# 시각화 패키지
import matplotlib.pyplot as plt # matplotlib 시각화 패키지
import seaborn as sns
%matplotlib inline

# os에 따른 Font 깨짐 제거를 위해 Font 지정
import os
if os.name == 'nt':
    font_family = "Malgun Gothic"
else:
    font_family = "AppleGothic"

# - 값이 깨지는 문제 해결을 위해 파라미터값 설정
sns.set(font=font_family, rc={"axes.unicode_minus": False})

## 1. 일표본 t-검정(one sample t-test)

In [3]:
# 모집단 데이터
data = [177.3, 182.7, 169.6, 160, 180.3, 179.4, 178.5, 177.2, 181.8, 176.5, 190, 185, 167, 171]

In [4]:
### 데이터의 정규성 검정: pvalue > 0.05이면 정규성을 따름
stats.shapiro(data)

ShapiroResult(statistic=0.9641273021697998, pvalue=0.7900116443634033)

stas.ttest_1samp로 검정 통계량 계산

In [5]:
# 유의수준 0.05
m_a = 0.05

# 알려진 평균 확인
s_mu = 175
one_sample_result = stats.ttest_1samp(data, popmean = s_mu)

print('t검정 통계량 = %.3f, pvalue = %.3f'%(one_sample_result))

t검정 통계량 = 0.903, pvalue = 0.383


In [6]:
# 검정통계량 직접 계산
# 표본 수
n = len(data)
print(len(data))
hysis_mu = np.mean(data)
print(np.mean(data))

# 알려진 평균
s_mu = 175

# 표본표준편차
s_sigma = np.std(data, ddof=1)
print(np.std(data, ddof=1))

# 통계량 t값 계산
t_value = (hysis_mu - s_mu) / (s_sigma / np.sqrt(n))

print('t검정 통계량 :', round(t_value, 3))

# p_value(유의확률) 계산
print('p_value(유의확률):', round((1 - stats.t(df=13).cdf(t_value) )*2, 3 ))

14
176.87857142857143
7.782443611837389
t검정 통계량 : 0.903
p_value(유의확률): 0.383


In [7]:
## 기각값
# 유의수준 0.05
print('유의수준: ', m_a)

# 기각값
t_95 = stats.t.ppf(1 - (1-0.95)/2, df=n-1)

print('기각값: ', t_95)

유의수준:  0.05
기각값:  2.1603686564610127


결론
- 검정 통계량 검정: 0.903은 채택역에 속함(기각역 2.16)
- P-value 검정: 0.05(유의수준) < 0.383(p-value), 귀무가설 채택
- 아이리포 중학교 1학년의 평균 키는 175cm이다.

## 2. 대응표본 t-검정 (paired sample t-test)
- stats.ttest_rel(before_data, after_data)

증명하고자 하는 가설
- 10명의 환자를 대상으로 비타민을 복용하기 전과 후의 수면시간을 측정하여 비타민의 효과가 있는지를 판단하고자 한다.
- 표본이 정규성을 만족한다는 가정하에 수면시간의 차이가 줄어 들었는지 검정(유의수준: 005)
- 귀무가설: 수면영양제를 복용하기 전과 후의 평균 수면시간에는 차이가 없다.(D=0)
- 대립가설: 수면영양제를 복용하기 전과 후의 평균 수면시간 차이는 0보다 작다.(D<0)

In [12]:
# 검정 데이터 관측치
before_data = [7,3,4,5,2,1,6,6,5,4]
after_data = [8,4,5,6,2,3,6,8,6,5]

check = ['before', 'after']
# 데이터프레임 merge
check = [check[j] for j in range(2) for i in range(10)]

data = pd.DataFrame({'when':check, 'score': before_data + after_data})

In [14]:
data.head()

,when,score
0,before,7
1,before,3
2,before,4
3,before,5
4,before,2


In [15]:
# 정규성 검정: 자료의 모집단 분포는 정규분포를 따름
from scipy.stats import shapiro

normal_before = shapiro(before_data)
normal_after = shapiro(after_data)

print(normal_before)
print(normal_after)

# p-value 모두 0.05보다 크기 때문에 정규성 만족

ShapiroResult(statistic=0.9644594192504883, pvalue=0.8352694511413574)
ShapiroResult(statistic=0.9456835985183716, pvalue=0.6177982091903687)


In [16]:
# 등분산성 검정: 모든 집단의 모분산은 동일함
from scipy.stats import levene
print(levene(before_data, after_data))

from scipy.stats import bartlett
print(bartlett(before_data, after_data))

# p-value가 0.05보다 커서 등분산성 만족

LeveneResult(statistic=0.0, pvalue=1.0)
BartlettResult(statistic=0.007785808167159078, pvalue=0.9296881301038368)


In [17]:
# 대응표본 t검정 수행
paired_result = stats.ttest_rel(before_data, after_data)

In [18]:
print('t검정 통계량 = %.3f, pvalue = %.3f'%paired_result)

t검정 통계량 = -4.743, pvalue = 0.001


## 3. 독립표본 t-검정 (Independent Sample t-Test)
- 등분산성 만족: stats.ttest_ind(x,y, equal_var=True)
- 등분산성 불만족: stats.ttest_ind(x,y,equal_var+False)

증명하고자 하는 가설
- 서울과 구미, 두 지역의 겨울 낮 최고기온에 차이가 있는지를 알아보기 위해 12일 동안 두 지역의 낮 최고기온을 측정한 데이터로 독립표본 t검정을 수행해보자, (표본이 정규성을 만족한다는 가정, 양측검정 수행)
- 귀무가설: a,b 두 지역에 따른 겨울 낮 최고 기온은 차이가 없다.
- 대립가설: a,b 두 지역에 따른 겨울 낮 최고 기온은 차이가 있다.

In [22]:
# 데이터 프레임
seoul = [-1,0,3,4,1,3,3,1,1,3,2,4]
daegu = [6,6,8,8,11,11,10,8,8,9,7,10]
region = ['seoul']*12 + ['daegu']*12
data = pd.DataFrame({'region': region, 'temp': seoul + daegu})

In [23]:
data.head(3)

,region,temp
0,seoul,-1
1,seoul,0
2,seoul,3


In [24]:
## 정규성 검정: 자료의 모집단 분포는 정규분포를 따름

normal_seoul = shapiro(seoul)
normal_daegu = shapiro(daegu)
print(normal_seoul)
print(normal_daegu)

# 결과: p-value > 0.05, 정규성을 만족

ShapiroResult(statistic=0.9178412556648254, pvalue=0.2685365378856659)
ShapiroResult(statistic=0.9212772846221924, pvalue=0.2966691553592682)


In [25]:
## 등분산성 검정: 모든 집단의 모분산은 동일함

# levene 등분산성 검정
print(levene(seoul, daegu))

# bartlett 등분산성 검정
print(bartlett(seoul, daegu))

# 결과: p-value > 0.05, 등분산성 만족

LeveneResult(statistic=0.0, pvalue=1.0)
BartlettResult(statistic=0.07093015354702552, pvalue=0.7899872613272646)


In [26]:
# 독립표본 t검정
ttest_result = stats.ttest_ind(seoul, daegu, equal_var=True) # 등분산성 만족
# ttest_ind(a,b,equal_var=False) # 등분산성 불만족

print("t검정 통계량: %.3f, pvalue=%.15f"%(ttest_result))

t검정 통계량: -9.562, pvalue=0.000000002711180


결론
- 독립표본 t검정 수행결과, 검정 통계량 t값은 -9.562, 유의확률(p-value)은 0.0000000002711180
- P-value 검정: 0.05(유의수준) > 0.0000000002711180(p-value), 귀무가설 기각
- 서울과 구미, 두 지역의 겨울 낮 최고기온에는 통계적으로 유의한 차이가 존재한다는 결론을 내릴 수 있음